In [1]:
using TSSOS
using DynamicPolynomials
using CSV
using DataFrames
using Statistics
using Random

In [ ]:
#dis = CSV.read("distanceSet.csv", header=1);
#taxi=[1.38,0.77,2.52,1.15,1.2,3.53,1.0,2.2,0.95,1.42,1.7,2.77,0.57, 0.81,21.47, 1.13,1.19, 0.83, 0.93, 3.3,0.71, 1.04, 11.2, 0.72, 2.63, 0.82, 10.32, 3.51,7.25, 0.48, 2.5, 1.5, 0.4, 1.86, 2.34, 1.42,2.6,1.4, 1.02, 5.96, 0.84, 1.2, 9.3, 1.66,2.0, 2.55, 2.04, 17.87,0.6, 0.57];

In [2]:
num_sample=500
trip_distance=CSV.read("C:\\Users\\zhouq\\Documents\\GGUberSources\\2018_Yellow_Taxi_Trip_Data.csv",header=1,limit=num_sample,threaded=false)[5];

In [58]:
trip_distance

500-element Array{Float64,1}:
  0.72
  1.86
  2.04
  0.81
  0.82
  1.66
  0.57
 11.2
  1.02
  2.6
  3.53
  0.84
 21.47
  ⋮
  1.78
  1.9
  1.3
  0.3
  0.7
  1.88
 11.7
  3.52
  1.44
  2.1
  1.6
  0.6

In [5]:
# new formulation 3 10/02/21
# calculate 3 objective: intra-fair inter-fair customer-careF3_tssos(5,5,taxi[1:5],0.5,0.5,0)
function F3_tssos_21(icount,jcount,taxi,lam1,lam2,lam3)
    #icount=5 # the number of customers
    #jcount=5 # the number of drivers
    #female=[2,4]
    #male=[1,3,5]
    female=collect(2:2:jcount)
    male=collect(1:2:jcount)
    
    mu1=10
    mu2=10
    mu3=0

    #lam1=1
    #lam2=1
    #lam3=0
    
    dis=rand(icount,jcount)*0.5
    
    # set nc operators
    @polyvar u[1:jcount] M[1:icount*jcount]; #z[1:jcount]; 
    var=vcat(u,M);

    #l2=[z[j]-1+(u[j]+0.01)/1.01 for j in 1:jcount];
    #l3=[z[j]+1-(u[j]+0.01)/1.01 for j in 1:jcount];
    #l4=[b1-(1/length(female))*sum(z[j] for j in female),b1-(1/length(male))*sum(z[j] for j in male)];
    #l5=[b2-sum(M[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount) for i in 1:icount];
    l6=[1-sum(M[(i-1)*(jcount)+j] for i in 1:icount) for j in 1:jcount];
    #l7=[M[(i-1)*(jcount)+j]*(M[(i-1)*(jcount)+j]-1) for i in 1:icount for j in 1:jcount]; #equality #numeq=25
    l8=[u[j] for j in 1:jcount];
    l9=[M[(i-1)*(jcount)+j] for i in 1:icount for j in 1:jcount];

    obj=lam1*sum((1-(u[j]+0.01)/1.01)^2 for j in 1:jcount)+lam2*(sum((u[j]+0.01)/1.01 for j in female)-sum((u[j]+0.01)/1.01 for j in male))^2+lam3*sum(M[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount for i in 1:icount)+sum(mu1*(sum(M[(i-1)*(jcount)+j]*(taxi[i]-dis[i,j]) for i in 1:icount)-u[j])^2  for j in 1:jcount)+sum(mu2*(sum(M[(i-1)*(jcount)+j] for j in 1:jcount)-1)^2  for i in 1:icount)+sum(M[(i-1)*(jcount)+j]*(M[(i-1)*(jcount)+j]-1) for i in 1:icount for j in 1:jcount)

    # pop
    pop=vcat(obj,l6,u); #,l7,l8,l9

    d=1; # the relaxation order
    opt,sol,data=cs_tssos_first(pop,var,d,numeq=icount*jcount,TS="MD",solution=true);
    #opt,sol,data=cs_tssos_higher!(data,TS="MD")
    
    u_sol=sol[1:jcount]
    M_sol=sol[jcount+1:end]
    #intra5=sum((1-(u_sol[j]+0.01)/1.01)^2 for j in 1:jcount)
    
    mu=sum((u_sol[j]+0.01)/1.01 for j in 1:jcount)/jcount
    muf=sum((u_sol[j]+0.01)/1.01 for j in female)/length(female)
    mum=sum((u_sol[j]+0.01)/1.01 for j in male)/length(male)
    
    intra4=1/(2*mu*jcount^2)*sum(abs((u_sol[k]+0.01)/1.01-(u_sol[j]+0.01)/1.01) for k in 1:jcount for j in 1:jcount)
    care=sum(M_sol[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount for i in 1:icount)
    
    intra2=abs(1/jcount*sum(((u_sol[j]+0.01)/(1.01*mu))*log((u_sol[j]+0.01)/(1.01*mu)) for j in 1:jcount))
    intra3=abs(1/jcount*sum(log((1.01*mu)/(u_sol[j]+0.01)) for j in 1:jcount))
    
    inter1=abs((length(female)/jcount)*muf/mu*log(muf/mu)+(length(male)/jcount)*mum/mu*log(mum/mu))
    inter2=abs((length(female)/jcount)*log(mu/muf)+(length(male)/jcount)*log(mu/mum))
    inter3=abs(sum((u_sol[j]+0.01)/1.01 for j in female)-sum((u_sol[j]+0.01)/1.01 for j in male))
    
    return [intra2,intra3,intra4,inter1,inter2,inter3,care]
    #return [intra4,inter3,care]

end

F3_tssos_21 (generic function with 1 method)

In [6]:
F3_tssos_21(5,5,trip_distance[1:5],0.5,0.5,0)

------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 11              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve te

7-element Array{Float64,1}:
 6.913126894571677e-14
 6.93801649715258e-14
 2.0328335767148196e-7
 1.1402495868931139e-14
 1.1801164631050584e-14
 0.00990099466325324
 2.74731992402135

In [54]:
# for pareto optimal
icount=5 # the number of customers
jcount=5 # the number of drivers
repeat=100
lam1_range=0.5:0.1:1

#lam2_range=1:-0.1:0 #step 0.1
df_212 = DataFrame(Intra_Fair2=Float64[],Intra_Fair3=Float64[],Intra_Fair4=Float64[],Inter_Fair1=Float64[],Inter_Fair2=Float64[],Inter_Fair3=Float64[],Customer_Care=Float64[],lam1=Float64[],lam2=Float64[],lam3=Float64[])
trip=trip_distance[randperm(size(trip_distance)[1])[1:jcount]]
for lam1 in lam1_range
    #for lam2 in 0:0.1:(1-lam1)
    if lam1 == 1
        multi=size(lam1_range)[1]-1
    else
        multi=1
    end
    for r in 1:repeat*multi
        obj=F3_tssos_21(icount,jcount,trip,lam1,1-lam1,0) #1-lam1-lam2
        push!(df_212, push!(obj,lam1,1-lam1,0))
    end
    #end
end

#df[!,:lam2] .= -df[!,:lam1].+1
#df[!,:lam3] .= 0

#CSV.write("ParetoOptimal.csv", df,header=true)

------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 11              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve te

Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 366
Optimizer  - Cones                  : 1
Optimizer  - Scalar variables       : 12                conic                  : 12              
Optimizer  - Semi-definite variables: 52                scalarized             : 2129            
Factor     - setup time             : 0.00              dense det. time        : 0.00            
Factor     - ML order time          : 0.00              GP order time          : 0.00            
Factor     - nonzeros before factor : 4.07e+04          after factor           : 5.01e+04        
Factor     - dense dim.             : 0                 flops                  : 1.00e+07        
ITE PFEAS    DFEAS    GFEAS    PRSTATUS   POBJ              DOBJ              MU       TIME  
0   6.3e+01  1.0e+00  1.0e+00  0.00e+00   0.000000000e+00   0.000000000e+00   1.0e+00  0.00  
1   2.9e+01  4.5e-01  6.3e-01  -8.98e-01  -2.865046478e+00  -3.867043551e+00  4.5e-01  

------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 11              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max        

10  1.5e-05  2.0e-07  2.4e-10  1.00e+00   2.324545629e+01   2.324545548e+01   2.0e-07  0.08  
11  5.4e-07  7.2e-09  1.6e-12  1.00e+00   2.324542119e+01   2.324542116e+01   7.1e-09  0.13  
12  1.6e-08  1.4e-09  8.0e-15  1.00e+00   2.324542006e+01   2.324542005e+01   2.1e-10  0.14  
Optimizer terminated. Time: 0.14    

optimum = 23.24542005544019
------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 11              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker termi

5   2.0e+00  2.9e-02  2.0e-02  2.30e-01   2.919512143e+01   2.891869606e+01   2.9e-02  0.08  
6   9.7e-01  1.4e-02  4.9e-03  2.03e+00   1.944380098e+01   1.937268577e+01   1.4e-02  0.09  
7   1.0e-01  1.5e-03  1.6e-04  1.43e+00   2.244698954e+01   2.243947133e+01   1.5e-03  0.09  
8   5.5e-03  7.9e-05  1.9e-06  1.08e+00   2.234050225e+01   2.234013547e+01   7.9e-05  0.11  
9   2.2e-04  3.2e-06  1.5e-08  1.00e+00   2.233468054e+01   2.233466595e+01   3.2e-06  0.11  
10  9.7e-06  1.4e-07  1.4e-10  1.00e+00   2.233446956e+01   2.233446892e+01   1.4e-07  0.13  
11  2.8e-07  4.2e-09  6.9e-13  1.00e+00   2.233446070e+01   2.233446068e+01   4.0e-09  0.13  
12  6.5e-09  4.9e-09  2.4e-15  1.00e+00   2.233446045e+01   2.233446045e+01   9.3e-11  0.14  
Optimizer terminated. Time: 0.16    

optimum = 22.334460453578664
------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  N

...

1   3.0e+01  4.5e-01  6.3e-01  -9.04e-01  -2.870578293e+00  -3.872657420e+00  4.5e-01  0.05  
2   1.7e+01  2.5e-01  4.2e-01  -7.68e-01  -6.440518502e+01  -6.630857670e+01  2.5e-01  0.08  
3   8.0e+00  1.2e-01  2.1e-01  -5.01e-01  -8.865416413e+01  -9.115695933e+01  1.2e-01  0.11  
4   6.0e+00  9.1e-02  1.5e-01  2.54e-01   1.854640364e+01   1.622817666e+01   9.1e-02  0.13  
5   2.1e+00  3.2e-02  2.2e-02  3.76e-01   2.066184614e+01   2.035592301e+01   3.2e-02  0.16  
6   8.8e-01  1.3e-02  3.9e-03  1.93e+00   1.116619654e+01   1.112136615e+01   1.3e-02  0.17  
7   7.0e-02  1.1e-03  8.5e-05  1.40e+00   1.269045799e+01   1.268675136e+01   1.1e-03  0.19  
8   1.9e-03  2.9e-05  3.7e-07  1.05e+00   1.266740040e+01   1.266730648e+01   2.9e-05  0.20  
9   2.9e-05  4.3e-07  6.9e-10  1.00e+00   1.266780472e+01   1.266780330e+01   4.3e-07  0.22  
10  9.2e-07  1.4e-08  3.9e-12  1.00e+00   1.266781208e+01   1.266781203e+01   1.4e-08  0.23  
11  2.9e-08  4.3e-09  2.2e-14  1.00e+00   1.266781210e+01   

------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 11              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max        

4   5.7e+00  1.0e-01  1.8e-01  1.67e-01   1.898898464e+01   1.655920802e+01   1.0e-01  0.06  
5   1.7e+00  3.1e-02  2.0e-02  2.14e-01   3.275070025e+01   3.249585718e+01   3.1e-02  0.09  
6   8.1e-01  1.5e-02  5.2e-03  1.97e+00   2.272276339e+01   2.264976487e+01   1.5e-02  0.14  
7   1.0e-01  1.9e-03  2.4e-04  1.42e+00   2.490811064e+01   2.489773960e+01   1.9e-03  0.17  
8   7.3e-03  1.3e-04  4.3e-06  1.08e+00   2.469222580e+01   2.469156435e+01   1.3e-04  0.20  
9   2.5e-04  4.6e-06  2.8e-08  1.01e+00   2.467879429e+01   2.467877170e+01   4.6e-06  0.22  
10  4.9e-06  9.1e-08  7.5e-11  1.00e+00   2.467836118e+01   2.467836074e+01   9.1e-08  0.25  
11  1.1e-07  2.4e-09  2.4e-13  1.00e+00   2.467835145e+01   2.467835145e+01   2.0e-09  0.27  
Optimizer terminated. Time: 0.28    

optimum = 24.67835145474412
------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Na

Excessive output truncated after 524291 bytes.


Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 11              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimiza

In [56]:
df_212

,Intra_Fair2,Intra_Fair3,Intra_Fair4,Inter_Fair1,Inter_Fair2,Inter_Fair3,Customer_Care
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.57526e-14,1.60886e-14,9.7198e-8,2.92213e-15,3.22851e-15,0.00990099,5.85701
2,1.85385e-14,1.90092e-14,1.06137e-7,5.20798e-16,1.05572e-15,0.00990099,5.16131
3,9.31724e-16,1.11109e-15,2.44024e-8,4.65196e-17,1.13133e-16,0.00990099,5.08419
4,1.60269e-13,1.60385e-13,3.06659e-7,6.2352e-15,6.1993e-15,0.00990099,6.10888
5,5.32991e-15,5.08398e-15,5.63177e-8,8.76203e-16,7.00314e-16,0.00990099,5.96047
6,2.82462e-13,2.82575e-13,4.05605e-7,5.80875e-14,5.8064e-14,0.00990099,6.10725
7,4.6518e-14,4.63633e-14,1.63931e-7,1.14433e-15,8.76275e-16,0.00990099,6.03764
8,2.41513e-15,2.62529e-15,3.81497e-8,3.26936e-16,4.94629e-16,0.00990099,5.66213
9,2.36042e-15,2.36913e-15,3.46103e-8,1.30381e-16,1.7479e-16,0.00990099,6.30486


In [55]:
CSV.write("ParetoOptimal_212.csv", df_212,header=true)

"ParetoOptimal_212.csv"

# Random trip distance for 5 runs

In [32]:
fold=5
# for pareto optimal
icount=5 # the number of customers
jcount=5 # the number of drivers
repeat=50
lam1_range=0.5:0.1:1

for f in 1:fold
    #lam2_range=1:-0.1:0 #step 0.1
    df_212R = DataFrame(Intra_Fair2=Float64[],Intra_Fair3=Float64[],Intra_Fair4=Float64[],Inter_Fair1=Float64[],Inter_Fair2=Float64[],Inter_Fair3=Float64[],Customer_Care=Float64[],lam1=Float64[],lam2=Float64[],lam3=Float64[])
    trip=trip_distance[randperm(size(trip_distance)[1])[1:jcount]]
    for lam1 in lam1_range
        #for lam2 in 0:0.1:(1-lam1)
        if lam1 == 1
            multi=size(lam1_range)[1]-1
        else
            multi=1
        end
        for r in 1:repeat*multi
            obj=F3_tssos_21(icount,jcount,trip,lam1,1-lam1,0) #1-lam1-lam2
            push!(df_212R, push!(obj,lam1,1-lam1,0))
        end
        #end
    end
    filename=string("ParetoOptimal_R",f,".csv")
    CSV.write(filename, df_212R,header=true)
end

------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 11              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve te

Factor     - nonzeros before factor : 4.07e+04          after factor           : 5.01e+04        
Factor     - dense dim.             : 0                 flops                  : 1.00e+07        
ITE PFEAS    DFEAS    GFEAS    PRSTATUS   POBJ              DOBJ              MU       TIME  
0   2.0e+02  1.0e+00  1.0e+00  0.00e+00   0.000000000e+00   0.000000000e+00   1.0e+00  0.03  
1   9.0e+01  4.6e-01  6.5e-01  -9.37e-01  -2.894226282e+00  -3.927165642e+00  4.6e-01  0.03  
2   5.2e+01  2.6e-01  4.6e-01  -8.42e-01  -7.450773757e+01  -7.663032457e+01  2.6e-01  0.05  
3   1.5e+01  7.5e-02  1.8e-01  -7.09e-01  -1.762772826e+02  -1.816108601e+02  7.5e-02  0.05  
4   7.9e+00  4.0e-02  6.9e-02  1.66e-01   -6.029176393e+00  -8.594082674e+00  4.0e-02  0.06  
5   2.6e+00  1.3e-02  6.9e-03  9.04e-01   1.753233164e+01   1.735420575e+01   1.3e-02  0.06  
6   5.5e-01  2.8e-03  6.8e-04  1.98e+00   1.594360749e+01   1.589592054e+01   2.8e-03  0.06  
7   2.3e-02  1.2e-04  5.2e-06  1.14e+00   1.86356986

7   3.5e-02  2.0e-04  8.8e-06  1.21e+00   1.740513001e+01   1.740398521e+01   2.0e-04  0.06  
8   1.1e-03  6.3e-06  4.8e-08  1.02e+00   1.754281129e+01   1.754277675e+01   6.3e-06  0.06  
9   4.9e-05  2.8e-07  4.6e-10  1.00e+00   1.754700623e+01   1.754700469e+01   2.8e-07  0.08  
10  2.2e-06  1.3e-08  4.4e-12  1.00e+00   1.754717972e+01   1.754717965e+01   1.3e-08  0.08  
11  1.2e-07  8.1e-10  5.2e-14  1.00e+00   1.754718668e+01   1.754718667e+01   6.7e-10  0.08  
Optimizer terminated. Time: 0.08    

optimum = 17.54718667798862
------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 11              
  Ma

optimum = 20.318540536284747
------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 11              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 :

optimum = 17.88972389281631
------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 11              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 

...

optimum = 18.522384966161326
------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 11              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 :

1   9.1e+01  4.6e-01  6.5e-01  -9.36e-01  -2.887094614e+00  -3.913793264e+00  4.6e-01  0.03  
2   5.2e+01  2.6e-01  4.5e-01  -8.34e-01  -7.309370810e+01  -7.520443324e+01  2.6e-01  0.05  
3   1.5e+01  7.8e-02  1.9e-01  -6.93e-01  -1.665036596e+02  -1.715167115e+02  7.8e-02  0.06  
4   9.1e+00  4.6e-02  8.3e-02  1.75e-01   1.254690711e+00   -1.583876574e+00  4.6e-02  0.06  
5   3.1e+00  1.6e-02  8.7e-03  7.75e-01   2.002537821e+01   1.981820158e+01   1.6e-02  0.06  
6   8.0e-01  4.0e-03  1.0e-03  1.99e+00   1.605135638e+01   1.600255594e+01   4.0e-03  0.08  
7   3.1e-02  1.6e-04  6.6e-06  1.17e+00   1.964657981e+01   1.964545148e+01   1.6e-04  0.08  
8   9.9e-04  5.0e-06  3.7e-08  1.01e+00   1.978343328e+01   1.978339829e+01   5.0e-06  0.08  
9   5.2e-05  2.6e-07  4.4e-10  1.00e+00   1.978763245e+01   1.978763062e+01   2.6e-07  0.09  
10  3.0e-06  1.5e-08  6.1e-12  1.00e+00   1.978784231e+01   1.978784220e+01   1.5e-08  0.09  
11  1.3e-07  7.0e-10  5.2e-14  1.00e+00   1.978785378e+01   

8   1.4e-03  7.9e-06  7.6e-08  1.01e+00   2.013750002e+01   2.013743728e+01   7.9e-06  0.09  
9   8.5e-05  4.7e-07  1.1e-09  1.00e+00   2.014428391e+01   2.014428023e+01   4.7e-07  0.11  
10  4.7e-06  2.6e-08  1.4e-11  1.00e+00   2.014466591e+01   2.014466571e+01   2.6e-08  0.11  
11  2.4e-07  1.6e-09  1.6e-13  1.00e+00   2.014468581e+01   2.014468580e+01   1.3e-09  0.11  
Optimizer terminated. Time: 0.11    

optimum = 20.14468581138074
------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 11              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer

Excessive output truncated after 524306 bytes.


Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 11              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimiza

# Random trip distance for each run | No compare

In [38]:
# for pareto optimal
icount=5 # the number of customers
jcount=5 # the number of drivers
repeat=1
lam1=0.5
num_trials=100
#lam2_range=1:-0.1:0 #step 0.1
df_random = DataFrame(Intra_Fair2=Float64[],Intra_Fair3=Float64[],Intra_Fair4=Float64[],Inter_Fair1=Float64[],Inter_Fair2=Float64[],Inter_Fair3=Float64[],Customer_Care=Float64[],Trial=Int64[])

for trial in 1:num_trials
    trip=trip_distance[randperm(size(trip_distance)[1])[1:jcount]]
    for r in 1:repeat
        obj=F3_tssos_21(icount,jcount,trip,lam1,1-lam1,0) #1-lam1-lam2
        push!(df_random, push!(obj,trial))
    end
end

------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 11              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve te

5   2.1e+00  2.5e-02  1.6e-02  2.89e-01   3.236772697e+01   3.210800132e+01   2.5e-02  0.09  
6   7.0e-01  8.4e-03  2.2e-03  2.13e+00   2.729309446e+01   2.725764028e+01   8.4e-03  0.13  
7   5.4e-02  6.5e-04  4.8e-05  1.28e+00   3.177849730e+01   3.177503668e+01   6.5e-04  0.14  
8   1.3e-03  1.5e-05  1.7e-07  1.04e+00   3.181341831e+01   3.181334002e+01   1.5e-05  0.16  
9   2.6e-05  3.2e-07  5.1e-10  1.00e+00   3.181372159e+01   3.181371994e+01   3.2e-07  0.17  
10  6.1e-07  7.4e-09  1.8e-12  1.00e+00   3.181373820e+01   3.181373816e+01   7.4e-09  0.17  
11  2.4e-08  3.0e-09  1.4e-14  1.00e+00   3.181373905e+01   3.181373904e+01   2.9e-10  0.19  
Optimizer terminated. Time: 0.20    

optimum = 31.81373904518564
------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type    

7   3.0e-03  4.5e-05  9.1e-07  1.07e+00   9.467282735e+00   9.466973706e+00   4.5e-05  0.11  
8   1.1e-04  1.7e-06  6.4e-09  1.00e+00   9.480800771e+00   9.480789584e+00   1.7e-06  0.14  
9   8.1e-06  1.2e-07  1.2e-10  1.00e+00   9.481125224e+00   9.481124434e+00   1.2e-07  0.14  
10  5.5e-07  8.2e-09  2.2e-12  1.00e+00   9.481143630e+00   9.481143577e+00   8.2e-09  0.16  
11  3.4e-08  3.0e-09  3.3e-14  1.00e+00   9.481144819e+00   9.481144816e+00   5.0e-10  0.17  
Optimizer terminated. Time: 0.17    

optimum = 9.481144819369893
------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 11              
  Ma

6   5.9e+00  2.8e-03  8.4e-03  1.60e+00   -2.619490944e+02  -2.711557508e+02  2.8e-03  0.09  
7   1.7e+00  8.0e-04  6.5e-04  1.60e+00   -2.347830143e+01  -2.410138809e+01  8.0e-04  0.13  
8   6.1e-02  2.9e-05  3.9e-06  1.43e+00   2.242498045e+01   2.240699156e+01   2.9e-05  0.14  
9   3.2e-04  1.5e-07  1.5e-09  1.02e+00   2.330934863e+01   2.330925422e+01   1.5e-07  0.14  
10  2.1e-05  1.0e-08  2.5e-11  1.00e+00   2.331464789e+01   2.331464168e+01   1.0e-08  0.16  
11  1.1e-06  5.1e-10  2.9e-13  1.00e+00   2.331506213e+01   2.331506182e+01   5.1e-10  0.17  
12  5.3e-08  3.6e-11  3.1e-15  1.00e+00   2.331508513e+01   2.331508511e+01   2.5e-11  0.19  
Optimizer terminated. Time: 0.20    

optimum = 23.315085126858175
------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type   

  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 366
Optimizer  - Cones                  : 1
Optimizer  - Scalar variables       : 12                conic                  : 12              
Optimizer  - Semi-definite variables: 52                scalarized             : 2129            
Factor     - setup time             : 0.02              dense det. time        : 0.00            
Factor     - ML order time          : 0.00              GP order time          : 0.00            
Factor     - nonzeros before factor : 4.07e+04          after factor           : 5.01e+04        
Factor     - dense dim.             : 0                 flops                  : 1.00e+07        
ITE PFEAS    DFEAS    GFEAS    PRSTATUS   POBJ              DOBJ              MU       TIME  
0   5.1e+02  1.0e+00  1.0e+00  0.00e+00   0.000000000e+00   0.000000000e+00   1.0

...

5   1.8e+00  2.6e-02  1.2e-02  6.08e-01   2.207467476e+01   2.199775721e+01   2.6e-02  0.08  
6   4.9e-01  7.1e-03  1.5e-03  2.04e+00   1.642375162e+01   1.639664458e+01   7.1e-03  0.09  
7   1.9e-02  2.7e-04  9.1e-06  1.20e+00   1.912404445e+01   1.912357803e+01   2.7e-04  0.11  
8   6.3e-04  9.0e-06  5.4e-08  1.01e+00   1.916857721e+01   1.916856194e+01   9.0e-06  0.14  
9   1.8e-05  2.5e-07  2.6e-10  1.00e+00   1.916998273e+01   1.916998230e+01   2.5e-07  0.16  
10  4.4e-07  6.4e-09  1.0e-12  1.00e+00   1.917002467e+01   1.917002466e+01   6.3e-09  0.19  
11  1.8e-08  2.4e-09  8.5e-15  1.00e+00   1.917002573e+01   1.917002573e+01   2.6e-10  0.20  
Optimizer terminated. Time: 0.20    

optimum = 19.170025732449737
------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type   

4   5.0e+00  2.9e-02  4.8e-02  3.76e-01   2.933866198e+01   2.696701800e+01   2.9e-02  0.03  
5   1.8e+00  1.0e-02  6.3e-03  1.25e+00   5.170175308e+01   5.141278856e+01   1.0e-02  0.08  
6   4.0e-01  2.3e-03  5.9e-04  1.64e+00   5.775035934e+01   5.769738469e+01   2.3e-03  0.09  
7   1.1e-02  6.4e-05  2.3e-06  1.16e+00   6.029010865e+01   6.028911261e+01   6.4e-05  0.09  
8   7.9e-05  4.6e-07  1.4e-09  1.01e+00   6.034367162e+01   6.034366431e+01   4.6e-07  0.11  
9   2.1e-06  1.2e-08  6.1e-12  1.00e+00   6.034396245e+01   6.034396226e+01   1.2e-08  0.13  
10  1.1e-07  7.7e-10  7.8e-14  1.00e+00   6.034396940e+01   6.034396939e+01   6.7e-10  0.13  
Optimizer terminated. Time: 0.13    

optimum = 60.34396939647583
------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type    

Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 11              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 366
Optimizer  - Cones                  : 1
Optimizer  - Scalar variables       : 12                conic                  : 12              
Optimizer  - Semi-definite variables: 52                scalarized             : 2129            
Factor     - setup time             : 0.00              dense det. time        : 0.00            
Factor     - ML order time          : 0.00  

1   1.7e+01  4.5e-01  6.1e-01  -8.52e-01  -2.624892254e+00  -3.569646429e+00  4.5e-01  0.06  
2   1.0e+01  2.8e-01  4.1e-01  -6.68e-01  -5.767414259e+01  -5.911080915e+01  2.8e-01  0.08  
3   4.4e+00  1.2e-01  1.5e-01  -3.16e-01  -5.071005864e+01  -5.198431520e+01  1.2e-01  0.09  
4   2.9e+00  7.7e-02  8.9e-02  6.40e-01   1.354077983e+01   1.256231022e+01   7.7e-02  0.11  
5   1.0e+00  2.6e-02  1.1e-02  1.16e+00   1.339018766e+01   1.330783486e+01   2.6e-02  0.13  
6   2.6e-01  7.0e-03  1.2e-03  1.76e+00   9.233917090e+00   9.216257518e+00   7.0e-03  0.14  
7   1.3e-02  3.5e-04  1.3e-05  1.17e+00   1.004902857e+01   1.004841079e+01   3.5e-04  0.16  
8   3.2e-04  8.4e-06  4.7e-08  1.01e+00   1.009023733e+01   1.009022228e+01   8.4e-06  0.17  
9   6.4e-06  1.7e-07  1.3e-10  1.00e+00   1.009136717e+01   1.009136687e+01   1.7e-07  0.17  
10  2.9e-07  7.7e-09  1.3e-12  1.00e+00   1.009138735e+01   1.009138733e+01   7.6e-09  0.19  
Optimizer terminated. Time: 0.19    

optimum = 10.091387346


Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.02            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.03    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 11              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints          

In [39]:
CSV.write("ParetoOptimal_random.csv", df_random,header=true)

"ParetoOptimal_random.csv"

# Old Formulation

In [36]:
F3_tssos_21(5,5,taxi[1:5],0.5,0.5,0)

------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 6               
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve te

7-element Array{Float64,1}:
 0.010421537623723588
 0.010371159305920988
 0.07875108426353188
 0.009604556491216623
 0.009487450682511084
 0.10312807654473781
 1.9073303474364536

In [8]:
# new formulation 3 19/11
# calculate 3 objective: intra-fair inter-fair customer-care
function F3_tssos(icount,jcount,taxi,lam1,lam2,lam3)
    #icount=5 # the number of customers
    #jcount=5 # the number of drivers
    #female=[2,4]
    #male=[1,3,5]
    female=collect(2:2:jcount)
    male=collect(1:2:jcount)
    
    mu1=10
    mu2=10
    mu3=0

    #lam1=1
    #lam2=1
    #lam3=0
    
    dis=rand(icount,jcount)
    
    # set nc operators
    @polyvar u[1:jcount] M[1:icount*jcount]; #z[1:jcount]; 
    var=vcat(u,M);

    #l2=[z[j]-1+(u[j]+0.01)/1.01 for j in 1:jcount];
    #l3=[z[j]+1-(u[j]+0.01)/1.01 for j in 1:jcount];
    #l4=[b1-(1/length(female))*sum(z[j] for j in female),b1-(1/length(male))*sum(z[j] for j in male)];
    #l5=[b2-sum(M[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount) for i in 1:icount];
    l6=[1-sum(M[(i-1)*(jcount)+j] for i in 1:icount) for j in 1:jcount];
    l7=[M[(i-1)*(jcount)+j]*(M[(i-1)*(jcount)+j]-1) for i in 1:icount for j in 1:jcount]; #equality #numeq=25

    obj=lam1*sum((1-(u[j]+0.01)/1.01)^2 for j in 1:jcount)+lam2*(sum((u[j]+0.01)/1.01 for j in female)-sum((u[j]+0.01)/1.01 for j in male))^2+lam3*sum(M[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount for i in 1:icount)+sum(mu1*(sum(M[(i-1)*(jcount)+j]*(taxi[i]-dis[i,j]) for i in 1:icount)-u[j])^2  for j in 1:jcount)+sum(mu2*(sum(M[(i-1)*(jcount)+j] for j in 1:jcount)-1)^2  for i in 1:icount)

    # pop
    pop=vcat(obj,l6,l7);

    d=1; # the relaxation order
    opt,sol,data=cs_tssos_first(pop,var,d,numeq=icount*jcount,TS="MD",solution=true);
    #opt,sol,data=cs_tssos_higher!(data,TS="MD")
    
    u_sol=sol[1:jcount]
    M_sol=sol[jcount+1:end]
    #intra5=sum((1-(u_sol[j]+0.01)/1.01)^2 for j in 1:jcount)
    
    mu=sum((u_sol[j]+0.01)/1.01 for j in 1:jcount)/jcount
    muf=sum((u_sol[j]+0.01)/1.01 for j in female)/length(female)
    mum=sum((u_sol[j]+0.01)/1.01 for j in male)/length(male)
    
    intra4=1/(2*mu*jcount^2)*sum(abs((u_sol[k]+0.01)/1.01-(u_sol[j]+0.01)/1.01) for k in 1:jcount for j in 1:jcount)
    inter3=(sum((u_sol[j]+0.01)/1.01 for j in female)-sum((u_sol[j]+0.01)/1.01 for j in male))^2
    care=sum(M_sol[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount for i in 1:icount)
    
    intra2=abs(1/jcount*sum(((u_sol[j]+0.01)/(1.01*mu))*log((u_sol[j]+0.01)/(1.01*mu)) for j in 1:jcount))
    intra3=abs(1/jcount*sum(log((1.01*mu)/(u_sol[j]+0.01)) for j in 1:jcount))
    
    inter1=abs((length(female)/jcount)*muf/mu*log(muf/mu)+(length(male)/jcount)*mum/mu*log(mum/mu))
    inter2=abs((length(female)/jcount)*log(mu/muf)+(length(male)/jcount)*log(mu/mum))
    
    return [intra2,intra3,intra4,inter1,inter2,inter3,care]
    #return [intra4,inter3,care]

end

LoadError: UndefVarError: df_run not defined

In [33]:
F3_tssos(5,5,taxi[1:5],0.5,0.5,0)

------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve te

7-element Array{Float64,1}:
 0.014244166433593611
 0.013803539392723739
 0.08636167013304498
 0.012512908004531934
 0.012351782298364561
 0.04063770364833207
 2.5180908064486256

In [34]:
# for pareto optimal
icount=5 # the number of customers
jcount=5 # the number of drivers
repeat=10
lam1_range=0:0.1:1  
#lam2_range=1:-0.1:0 #step 0.1
df = DataFrame(Intra_Fair2=Float64[],Intra_Fair3=Float64[],Intra_Fair4=Float64[],Inter_Fair1=Float64[],Inter_Fair2=Float64[],Inter_Fair3=Float64[],Customer_Care=Float64[],lam1=Float64[],lam2=Float64[],lam3=Float64[])

for lam1 in lam1_range
    for lam2 in 0:0.1:(1-lam1)
        for r in 1:repeat
            obj=F3_tssos(icount,jcount,taxi[1:5],lam1,lam2,1-lam1-lam2)
            push!(df, push!(obj,lam1,lam2,1-lam1-lam2))
        end
    end
end

#df[!,:lam2] .= -df[!,:lam1].+1
#df[!,:lam3] .= 0

#CSV.write("ParetoOptimal.csv", df,header=true)

------------------------------------------------------
The clique sizes of varibles:
[16, 14, 9, 6]
[1, 2, 5, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 296             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 57              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve term

5   2.3e+00  4.0e-02  2.6e-02  2.20e+00   -4.804706973e+01  -4.835370686e+01  4.0e-02  0.11  
6   4.2e-01  7.3e-03  1.6e-03  1.59e+00   -4.905767967e+00  -4.938222758e+00  7.3e-03  0.11  
7   8.4e-02  1.5e-03  1.4e-04  1.09e+00   4.507790247e-01   4.452649178e-01   1.5e-03  0.13  
8   2.6e-02  4.4e-04  2.1e-05  1.00e+00   8.208814886e-01   8.195767808e-01   4.4e-04  0.14  
9   6.8e-03  1.2e-04  2.8e-06  1.01e+00   9.752981361e-01   9.750056740e-01   1.2e-04  0.16  
10  1.3e-03  2.2e-05  2.2e-07  1.00e+00   9.940239078e-01   9.939748042e-01   2.2e-05  0.16  
11  4.5e-04  7.8e-06  4.4e-08  1.00e+00   1.000316949e+00   1.000300889e+00   7.8e-06  0.17  
12  1.7e-04  3.0e-06  1.1e-08  9.99e-01   1.002386046e+00   1.002379881e+00   3.0e-06  0.19  
13  3.2e-05  5.6e-07  8.3e-10  9.99e-01   1.003594402e+00   1.003593405e+00   5.6e-07  0.20  
14  5.5e-06  9.4e-08  5.9e-11  1.00e+00   1.003967076e+00   1.003966891e+00   9.4e-08  0.20  
15  4.2e-07  7.4e-09  1.3e-12  1.00e+00   1.004035021e+00   

optimum = 1.7098866990150885
------------------------------------------------------
The clique sizes of varibles:
[16, 14, 9, 6]
[1, 2, 5, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 296             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 57              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0

Factor     - ML order time          : 0.00              GP order time          : 0.00            
Factor     - nonzeros before factor : 2.17e+04          after factor           : 2.67e+04        
Factor     - dense dim.             : 0                 flops                  : 3.85e+06        
ITE PFEAS    DFEAS    GFEAS    PRSTATUS   POBJ              DOBJ              MU       TIME  
0   5.8e+01  1.0e+00  1.0e+00  0.00e+00   0.000000000e+00   0.000000000e+00   1.0e+00  0.03  
1   4.0e+01  6.9e-01  8.1e-01  -8.83e-01  -2.782754674e+00  -3.183702811e+00  6.9e-01  0.11  
2   3.2e+01  5.5e-01  7.1e-01  -9.27e-01  -4.157971232e+01  -4.229409429e+01  5.5e-01  0.14  
3   1.1e+01  1.9e-01  3.8e-01  -8.74e-01  -2.091878371e+02  -2.123521600e+02  1.9e-01  0.16  
4   4.3e+00  7.4e-02  9.0e-02  -3.35e-01  -2.102236033e+02  -2.113504327e+02  7.4e-02  0.19  
5   2.3e+00  3.9e-02  2.5e-02  2.19e+00   -4.613886701e+01  -4.642140344e+01  3.9e-02  0.22  
6   4.0e-01  6.8e-03  1.4e-03  1.58e+00   -4.410

4   5.0e+00  7.5e-02  9.1e-02  -3.34e-01  -2.063573200e+02  -2.074449827e+02  7.5e-02  0.13  
5   2.6e+00  3.9e-02  2.4e-02  2.16e+00   -4.511445081e+01  -4.537774893e+01  3.9e-02  0.14  
6   5.4e-01  8.1e-03  1.8e-03  1.58e+00   -5.437288443e+00  -5.468408894e+00  8.1e-03  0.16  
7   1.4e-01  2.1e-03  2.2e-04  1.10e+00   -3.790691331e-01  -3.859665139e-01  2.1e-03  0.19  
8   3.9e-02  5.8e-04  3.1e-05  1.03e+00   1.995359698e-01   1.979828526e-01   5.8e-04  0.20  
9   6.8e-03  1.0e-04  2.1e-06  1.02e+00   4.374581294e-01   4.372385435e-01   1.0e-04  0.22  
10  1.7e-03  2.6e-05  2.6e-07  9.93e-01   4.416922270e-01   4.416403546e-01   2.6e-05  0.25  
11  3.8e-04  5.8e-06  2.7e-08  9.99e-01   4.463560260e-01   4.463456388e-01   5.8e-06  0.27  
12  1.4e-04  2.1e-06  5.8e-09  9.99e-01   4.479603551e-01   4.479566524e-01   2.1e-06  0.28  
13  4.1e-05  6.2e-07  9.6e-10  1.00e+00   4.483789628e-01   4.483778214e-01   6.2e-07  0.30  
14  1.0e-05  1.5e-07  1.2e-10  1.00e+00   4.485448398e-01   

...

------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max        

4   3.9e+00  5.7e-02  3.8e-02  2.77e-03   -1.100970308e+02  -1.103040283e+02  5.7e-02  0.08  
5   1.0e+00  1.5e-02  3.9e-03  1.84e+00   -1.459888002e+01  -1.462793116e+01  1.5e-02  0.09  
6   2.0e-01  2.9e-03  3.2e-04  1.19e+00   -1.380775987e+00  -1.386190807e+00  2.9e-03  0.13  
7   4.6e-02  6.8e-04  3.7e-05  1.05e+00   7.371937948e-01   7.357851022e-01   6.8e-04  0.14  
8   1.4e-02  2.1e-04  6.4e-06  1.02e+00   9.070545919e-01   9.066116509e-01   2.1e-04  0.16  
9   3.7e-03  5.4e-05  8.3e-07  1.01e+00   9.847871247e-01   9.846698859e-01   5.4e-05  0.19  
10  8.7e-04  1.3e-05  9.4e-08  1.00e+00   1.000308827e+00   1.000281972e+00   1.3e-05  0.20  
11  1.7e-04  2.5e-06  8.2e-09  9.99e-01   1.005030577e+00   1.005025162e+00   2.5e-06  0.23  
12  3.7e-05  5.4e-07  8.2e-10  1.00e+00   1.005702597e+00   1.005701413e+00   5.4e-07  0.25  
13  3.6e-06  5.2e-08  2.4e-11  1.00e+00   1.005862055e+00   1.005861948e+00   5.2e-08  0.25  
14  1.8e-07  3.0e-09  2.8e-13  1.00e+00   1.005871071e+00   

optimum = 0.8974942002198315
------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 :

Excessive output truncated after 524288 bytes.


------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               


In [35]:
CSV.write("ParetoOptimal_1.csv", df,header=true)

"ParetoOptimal_1.csv"